# Interrater Reliability Evaluation: Krippendorff's alpha
### (Lasser et al., 2023)

In [1]:
# specify path for directory with data under DATA_PATH!

In [2]:
import pandas as pd
import numpy as np
import simpledorff

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# define global variables
DATA_PATH = '../../../data/labelled_samples'
CATEGORIES = ['[STRATEGY]', '[GROUP]', '[GOAL]', '[HATE]', '[TARGET]']

## Define evaluation functions

In [5]:
def load_data(file, annotator=''):
    # loads the data and adds column annotator_id
    data = pd.read_csv(DATA_PATH + file, sep=';')
    data = data.replace({np.NaN: 'NA'})
    data['annotator_id'] = [annotator] * len(data)
    return data

In [6]:
def krips_alpha(dfs=[], category='[STRATEGY]'):
    # calculates Krippendorff's alpha for all dataframes in dfs and category
    
    alpha_df = dfs[0]
    for i in range(len(dfs) - 1):
        alpha_df = pd.concat([alpha_df, dfs[i+1]])
        dups = alpha_df.duplicated(subset=['text'], keep=False)
        alpha_df = alpha_df[dups.values]

    if len(alpha_df) not in [100, 150, 1452, 1792, 3984]:
        # 100: two-way, 150: three-way, 1452: batch 1 STRATEGY, 1792: batch 1 GROUP GOAL HATE TARGET, 3984: batch 2
        raise AssertionError('Number of entries in concatenated dataframe not one of [100, 150, 1452, 1792, 3984].')
    try: 
        alpha = simpledorff.calculate_krippendorffs_alpha_for_df(alpha_df,
                                                                 experiment_col='text',
                                                                 annotator_col='annotator_id',
                                                                 class_col=category)
        return alpha
    except ZeroDivisionError:
        return np.nan

In [7]:
def retrieve_all_alphas(dfs=[], names=['LT', 'AS'], batch=''):
    # retrieves Krippendorff's alpha for all categories calling krips_alpha() and prints results
    print(f"BATCH {batch} // Krippendorff's alpha {'-'.join(names)}:")
    for category in CATEGORIES:
        print(category, '{:.2f}'.format(krips_alpha(dfs, category)))
    print('\n')

## Batch 1 // Development batch

In [8]:
data_LT = load_data('batch_1_LT.csv', 'LT')
data_AS = load_data('batch_1_AS.csv', 'AS')

In [9]:
# exclude first 270 Tweets for STRATEGY & batch 1 (because labeled together)
data_LT_STRATEGY = data_LT[270:]
data_LT_STRATEGY.drop_duplicates(subset='text', keep='first', inplace=True)
data_AS_STRATEGY = data_AS[270:]
data_AS_STRATEGY.drop_duplicates(subset='text', keep='first', inplace=True)

print("BATCH 1 // Krippendorff's alpha for STRATEGY:")
print('{:.2f}'.format(krips_alpha([data_LT_STRATEGY, data_AS_STRATEGY])))

BATCH 1 // Krippendorff's alpha for STRATEGY:
0.41


In [10]:
# exclude every 10th Tweet for GROUP, GOAL, HATE, TARGET & batch 1 (because labeled together)
data_LT_GROUP_GOAL_HATE = data_LT[data_LT.index % 10 != 0]
data_LT_GROUP_GOAL_HATE.drop_duplicates(subset='text', keep='first', inplace=True)
data_AS_GROUP_GOAL_HATE = data_AS[data_AS.index % 10 != 0]
data_AS_GROUP_GOAL_HATE.drop_duplicates(subset='text', keep='first', inplace=True)

CATEGORIES.pop(0)
retrieve_all_alphas([data_LT_GROUP_GOAL_HATE, data_AS_GROUP_GOAL_HATE], batch=1)
CATEGORIES = ['[STRATEGY]'] + CATEGORIES

BATCH 1 // Krippendorff's alpha LT-AS:
[GROUP] 0.50
[GOAL] 0.47
[HATE] 0.52
[TARGET] 0.53




## Batch 2 // Validation batch

In [11]:
data_LT = load_data('batch_2_LT.csv', 'LT')
data_LT.drop_duplicates(subset='text', keep='first', inplace=True)
data_AH = load_data('batch_2_AH.csv', 'AH')
data_AH.drop_duplicates(subset='text', keep='first', inplace=True)
data_AS = load_data('batch_2_AS.csv', 'AS')
data_AS.drop_duplicates(subset='text', keep='first', inplace=True)
data_EM = load_data('batch_2_EM.csv', 'EM')
data_EM.drop_duplicates(subset='text', keep='first', inplace=True)

In [12]:
retrieve_all_alphas([data_LT, data_AH, data_AS, data_EM], names=['LT', 'AH', 'AS', 'EM'], batch=2)

BATCH 2 // Krippendorff's alpha LT-AH-AS-EM:
[STRATEGY] 0.45
[GROUP] 0.42
[GOAL] 0.45
[HATE] 0.50
[TARGET] 0.58




## Batch 3 (including additional annotator conference with re-evaluation)

In [13]:
data_LT = load_data('batch_3_LT.csv', 'LT')
data_LT_AH = load_data('batch_3_LT_AH.csv', 'AH')

data_AH = load_data('batch_3_AH.csv', 'AH')
data_AH_LT = load_data('batch_3_AH_LT.csv', 'LT')
data_AH_LT_AS = load_data('batch_3_AH_LT_AS.csv', 'AS')

data_AS = load_data('batch_3_AS.csv', 'AS')
data_AS_AH = load_data('batch_3_AS_AH.csv', 'AH')
data_AS_AH_LT = load_data('batch_3_AS_AH_LT.csv', 'LT')

In [14]:
print('Based on batch LT\n')
print('* ')
retrieve_all_alphas(dfs=[data_LT, data_LT_AH], names=['LT', 'AH'], batch=3)

print('Based on batch AH\n')
print('* ')
retrieve_all_alphas(dfs=[data_AH, data_AH_LT], names=['AH', 'LT'], batch=3)
retrieve_all_alphas(dfs=[data_AH, data_AH_LT_AS], names=['AH', 'AS'], batch=3)
retrieve_all_alphas(dfs=[data_AH_LT, data_AH_LT_AS], names=['AS', 'LT'], batch=3)

print('Based on batch AS\n')
print('* ')
retrieve_all_alphas(dfs=[data_AS, data_AS_AH], names=['AS', 'AH'], batch=3)
retrieve_all_alphas(dfs=[data_AS, data_AS_AH_LT], names=['AS', 'LT'], batch=3)
retrieve_all_alphas(dfs=[data_AS_AH, data_AS_AH_LT], names=['AH', 'LT'], batch=3)

print('* indicates original cross-labeling analyses before annotator conference')

Based on batch LT

* 
BATCH 3 // Krippendorff's alpha LT-AH:
[STRATEGY] 0.42
[GROUP] 0.39
[GOAL] 0.29
[HATE] 0.61
[TARGET] 0.65


Based on batch AH

* 
BATCH 3 // Krippendorff's alpha AH-LT:
[STRATEGY] 0.48
[GROUP] 0.36
[GOAL] 0.40
[HATE] 0.45
[TARGET] 0.62


BATCH 3 // Krippendorff's alpha AH-AS:
[STRATEGY] 0.53
[GROUP] 0.30
[GOAL] 0.38
[HATE] 0.24
[TARGET] 0.41


BATCH 3 // Krippendorff's alpha AS-LT:
[STRATEGY] 0.40
[GROUP] 0.22
[GOAL] 0.39
[HATE] 0.00
[TARGET] 0.14


Based on batch AS

* 
BATCH 3 // Krippendorff's alpha AS-AH:
[STRATEGY] 0.16
[GROUP] 0.25
[GOAL] 0.31
[HATE] 0.12
[TARGET] 0.28


BATCH 3 // Krippendorff's alpha AS-LT:
[STRATEGY] 0.23
[GROUP] 0.22
[GOAL] 0.37
[HATE] 0.14
[TARGET] -0.03


BATCH 3 // Krippendorff's alpha AH-LT:
[STRATEGY] 0.24
[GROUP] 0.39
[GOAL] 0.32
[HATE] 0.18
[TARGET] 0.25


* indicates original cross-labeling analyses before annotator conference


In [15]:
print('Batch 3 AH three way')
retrieve_all_alphas([data_AH, data_AH_LT, data_AH_LT_AS], names=[], batch=3)
print('Batch 3 AS three way')
retrieve_all_alphas([data_AS, data_AS_AH, data_AS_AH_LT], names=[], batch=3)

Batch 3 AH three way
BATCH 3 // Krippendorff's alpha :
[STRATEGY] 0.47
[GROUP] 0.29
[GOAL] 0.39
[HATE] 0.24
[TARGET] 0.40


Batch 3 AS three way
BATCH 3 // Krippendorff's alpha :
[STRATEGY] 0.21
[GROUP] 0.29
[GOAL] 0.33
[HATE] 0.16
[TARGET] 0.20




## Batch 4 (including additional annotator conference with re-evaluation)

In [16]:
data_LT = load_data('batch_4_LT.csv', 'LT')
data_LT_AH = load_data('batch_4_LT_AH.csv', 'AH')
data_LT_AH_AS = load_data('batch_4_LT_AH_AS.csv', 'AS')

data_AH = load_data('batch_4_AH.csv', 'AH')
data_AH_AS = load_data('batch_4_AH_AS.csv', 'AS')
data_AH_AS_LT = load_data('batch_4_AH_AS_LT.csv', 'LT')

data_AS = load_data('batch_4_AS.csv', 'AS')
data_AS_AH = load_data('batch_4_AS_AH.csv', 'AH')
data_AS_AH_LT = load_data('batch_4_AS_AH_LT.csv', 'LT')

data_AS_AH_AS = load_data('batch_4_AS_AH_AS.csv', 'AS')

In [17]:
print('Based on batch LT\n')
print('* ')
retrieve_all_alphas(dfs=[data_LT, data_LT_AH], names=['LT', 'AH'], batch=4)
retrieve_all_alphas(dfs=[data_LT, data_LT_AH_AS], names=['LT', 'AS'], batch=4)
retrieve_all_alphas(dfs=[data_LT_AH, data_LT_AH_AS], names=['AH', 'AS'], batch=4)

print('Based on batch AH\n')
print('* ')
retrieve_all_alphas(dfs=[data_AH, data_AH_AS], names=['AH', 'AS'], batch=4)
retrieve_all_alphas(dfs=[data_AH, data_AH_AS_LT], names=['AH', 'LT'], batch=4)
retrieve_all_alphas(dfs=[data_AH_AS, data_AH_AS_LT], names=['AS', 'LT'], batch=4)

print('Based on batch AS\n')
print('OLD LABELS AS\n')
print('* ')
retrieve_all_alphas(dfs=[data_AS, data_AS_AH], names=['AS', 'AH'], batch=4)
retrieve_all_alphas(dfs=[data_AS, data_AS_AH_LT], names=['AS', 'LT'], batch=4)
retrieve_all_alphas(dfs=[data_AS_AH, data_AS_AH_LT], names=['AH', 'LT'], batch=4)

print('NEW LABELS AS\n')
print('* ')
retrieve_all_alphas(dfs=[data_AS_AH_AS, data_AS_AH], names=['AS', 'AH'], batch=4)
retrieve_all_alphas(dfs=[data_AS_AH_AS, data_AS_AH_LT], names=['AS', 'LT'], batch=4)

print('* indicates original cross-labeling analyses before annotator conference')

Based on batch LT

* 
BATCH 4 // Krippendorff's alpha LT-AH:
[STRATEGY] 0.43
[GROUP] 0.43
[GOAL] 0.43
[HATE] 0.53
[TARGET] 0.56


BATCH 4 // Krippendorff's alpha LT-AS:
[STRATEGY] 0.42
[GROUP] 0.28
[GOAL] 0.29
[HATE] 0.57
[TARGET] 0.61


BATCH 4 // Krippendorff's alpha AH-AS:
[STRATEGY] 0.43
[GROUP] 0.16
[GOAL] 0.41
[HATE] 0.46
[TARGET] 0.56


Based on batch AH

* 
BATCH 4 // Krippendorff's alpha AH-AS:
[STRATEGY] 0.25
[GROUP] 0.00
[GOAL] 0.07
[HATE] 0.31
[TARGET] 0.43


BATCH 4 // Krippendorff's alpha AH-LT:
[STRATEGY] 0.37
[GROUP] 0.28
[GOAL] 0.23
[HATE] 0.58
[TARGET] 0.62


BATCH 4 // Krippendorff's alpha AS-LT:
[STRATEGY] 0.24
[GROUP] 0.18
[GOAL] 0.14
[HATE] 0.41
[TARGET] 0.47


Based on batch AS

OLD LABELS AS

* 
BATCH 4 // Krippendorff's alpha AS-AH:
[STRATEGY] 0.24
[GROUP] 0.51
[GOAL] 0.44
[HATE] 0.18
[TARGET] 0.29


BATCH 4 // Krippendorff's alpha AS-LT:
[STRATEGY] 0.33
[GROUP] 0.40
[GOAL] 0.44
[HATE] 0.33
[TARGET] 0.41


BATCH 4 // Krippendorff's alpha AH-LT:
[STRATEGY] 0.36


In [18]:
print('Batch 4 LT three way')
retrieve_all_alphas([data_LT, data_LT_AH, data_LT_AH_AS], names=[], batch=3)
print('Batch 4 AH three way')
retrieve_all_alphas([data_AH, data_AH_AS, data_AH_AS_LT], names=[], batch=3)
print('Batch 4 AS three way (old labels AS)')
retrieve_all_alphas([data_AS, data_AS_AH, data_AS_AH_LT], names=[], batch=3)
print('Batch 4 AS three way (new labels AS)')
retrieve_all_alphas([data_AS_AH_AS, data_AS_AH, data_AS_AH_LT], names=[], batch=3)

Batch 4 LT three way
BATCH 3 // Krippendorff's alpha :
[STRATEGY] 0.43
[GROUP] 0.29
[GOAL] 0.38
[HATE] 0.52
[TARGET] 0.58


Batch 4 AH three way
BATCH 3 // Krippendorff's alpha :
[STRATEGY] 0.29
[GROUP] 0.15
[GOAL] 0.15
[HATE] 0.44
[TARGET] 0.51


Batch 4 AS three way (old labels AS)
BATCH 3 // Krippendorff's alpha :
[STRATEGY] 0.31
[GROUP] 0.45
[GOAL] 0.44
[HATE] 0.37
[TARGET] 0.45


Batch 4 AS three way (new labels AS)
BATCH 3 // Krippendorff's alpha :
[STRATEGY] 0.37
[GROUP] 0.43
[GOAL] 0.47
[HATE] 0.57
[TARGET] 0.64




## Batch 5

In [19]:
data_LT = load_data('batch_5_LT.csv', 'LT')
data_LT_AH = load_data('batch_5_LT_AH.csv', 'AH')

data_AH = load_data('batch_5_AH.csv', 'AH')
data_AH_LT = load_data('batch_5_AH_LT.csv', 'LT')

data_AS = load_data('batch_5_AS.csv', 'AS')
data_AS_AH = load_data('batch_5_AS_AH.csv', 'AH')

In [20]:
retrieve_all_alphas(dfs=[data_LT, data_LT_AH], names=['LT', 'AH'], batch=5)
retrieve_all_alphas(dfs=[data_AH, data_AH_LT], names=['AH', 'LT'], batch=5)
retrieve_all_alphas(dfs=[data_AS, data_AS_AH], names=['AS', 'AH'], batch=5)

BATCH 5 // Krippendorff's alpha LT-AH:
[STRATEGY] 0.53
[GROUP] 0.31
[GOAL] 0.24
[HATE] 0.47
[TARGET] 0.60


BATCH 5 // Krippendorff's alpha AH-LT:
[STRATEGY] 0.39
[GROUP] 0.35
[GOAL] 0.30
[HATE] 0.27
[TARGET] 0.41


BATCH 5 // Krippendorff's alpha AS-AH:
[STRATEGY] 0.58
[GROUP] 0.18
[GOAL] 0.36
[HATE] 0.11
[TARGET] 0.28




## Batch 6

In [21]:
data_LT = load_data('batch_6_LT.csv', 'LT')
data_LT_AH = load_data('batch_6_LT_AH.csv', 'AH')

data_AH = load_data('batch_6_AH.csv', 'AH')
data_AH_LT = load_data('batch_6_AH_LT.csv', 'LT')

In [22]:
# batch_6_AS contains 385 Tweets!
# Krippendorff's alpha calculated based on 35 overlapping Tweets
data_AS = load_data('batch_6_AS*.csv', 'AS')
data_AS.drop_duplicates(keep='first', inplace=True)
data_AS_AH = load_data('batch_6_AS_AH*.csv', 'AH')

In [23]:
retrieve_all_alphas(dfs=[data_LT, data_LT_AH], names=['LT', 'AH'], batch=6)
retrieve_all_alphas(dfs=[data_AH, data_AH_LT], names=['AH', 'LT'], batch=6)

BATCH 6 // Krippendorff's alpha LT-AH:
[STRATEGY] 0.26
[GROUP] 0.39
[GOAL] 0.17
[HATE] 0.43
[TARGET] 0.63


BATCH 6 // Krippendorff's alpha AH-LT:
[STRATEGY] 0.15
[GROUP] 0.29
[GOAL] 0.10
[HATE] 0.35
[TARGET] 0.35




In [24]:
alpha_df = pd.concat([data_AS, data_AS_AH])

dups = alpha_df.duplicated(subset=['text'], keep=False)
alpha_df = alpha_df[dups.values]

print("BATCH 6 // Krippendorff's Alpha AS-AH:")
for cat in CATEGORIES:
    alpha = simpledorff.calculate_krippendorffs_alpha_for_df(alpha_df,
                                                             experiment_col='text',
                                                             annotator_col='annotator_id',
                                                             class_col=cat)
    print(cat, '{:.2f}'.format(alpha))

BATCH 6 // Krippendorff's Alpha AS-AH:
[STRATEGY] 0.35
[GROUP] 0.30
[GOAL] 0.23
[HATE] 0.34
[TARGET] 0.40


## Batch 7

In [25]:
data_LTa = load_data('batch_7a_LT.csv', 'LT')
data_LTa_AH = load_data('batch_7a_LT_AH.csv', 'AH')

data_LTb = load_data('batch_7b_LT.csv', 'LT')
data_LTb_AH = load_data('batch_7b_LT_AH.csv', 'AH')

In [26]:
retrieve_all_alphas(dfs=[data_LTa, data_LTa_AH], names=['LT', 'AH'], batch=7)
retrieve_all_alphas(dfs=[data_LTb, data_LTb_AH], names=['LT', 'AH'], batch=7)

BATCH 7 // Krippendorff's alpha LT-AH:
[STRATEGY] 0.31
[GROUP] 0.20
[GOAL] 0.12
[HATE] 0.19
[TARGET] 0.34


BATCH 7 // Krippendorff's alpha LT-AH:
[STRATEGY] 0.28
[GROUP] 0.33
[GOAL] 0.24
[HATE] 0.25
[TARGET] 0.52




## Batch 8

In [27]:
data_LT = load_data('batch_8_LT.csv', 'LT')
data_LT_EM = load_data('batch_8_LT_EM.csv', 'EM')

data_EM = load_data('batch_8_EM.csv', 'EM')
data_EM_LT = load_data('batch_8_EM_LT.csv', 'LT')

In [28]:
retrieve_all_alphas(dfs=[data_LT, data_LT_EM], names=['LT', 'EM'], batch=8)
retrieve_all_alphas(dfs=[data_EM, data_EM_LT], names=['EM', 'LT'], batch=8)

BATCH 8 // Krippendorff's alpha LT-EM:
[STRATEGY] 0.42
[GROUP] 0.55
[GOAL] 0.33
[HATE] 0.27
[TARGET] 0.27


BATCH 8 // Krippendorff's alpha EM-LT:
[STRATEGY] 0.47
[GROUP] 0.32
[GOAL] 0.19
[HATE] 0.48
[TARGET] 0.34




## Batch 9

In [29]:
data_LT = load_data('batch_9_LT.csv', 'LT')
data_LT_EM = load_data('batch_9_LT_EM.csv', 'EM')

data_EM = load_data('batch_9_EM.csv', 'EM')
data_EM_LT = load_data('batch_9_EM_LT.csv', 'LT')

In [30]:
retrieve_all_alphas(dfs=[data_LT, data_LT_EM], names=['LT', 'EM'], batch=9)
retrieve_all_alphas(dfs=[data_EM, data_EM_LT], names=['EM', 'LT'], batch=9)

BATCH 9 // Krippendorff's alpha LT-EM:
[STRATEGY] 0.14
[GROUP] 0.31
[GOAL] 0.15
[HATE] 0.54
[TARGET] 0.72


BATCH 9 // Krippendorff's alpha EM-LT:
[STRATEGY] 0.39
[GROUP] 0.42
[GOAL] 0.34
[HATE] 0.59
[TARGET] 0.66




## Batch 10

In [31]:
data_LT = load_data('batch_10_LT.csv', 'LT')
data_LT_EM = load_data('batch_10_LT_EM.csv', 'EM')

data_EM = load_data('batch_10_EM.csv', 'EM')
data_EM_LT = load_data('batch_10_EM_LT.csv', 'LT')

In [32]:
retrieve_all_alphas(dfs=[data_LT, data_LT_EM], names=['LT', 'EM'], batch=10)
retrieve_all_alphas(dfs=[data_EM, data_EM_LT], names=['EM', 'LT'], batch=10)

BATCH 10 // Krippendorff's alpha LT-EM:
[STRATEGY] 0.42
[GROUP] 0.16
[GOAL] 0.23
[HATE] 0.49
[TARGET] 0.55


BATCH 10 // Krippendorff's alpha EM-LT:
[STRATEGY] 0.58
[GROUP] 0.34
[GOAL] 0.24
[HATE] 0.36
[TARGET] 0.49




## Batch 11

In [33]:
data_LT = load_data('batch_11_LT.csv', 'LT')
data_LT_EM = load_data('batch_11_LT_EM.csv', 'EM')

data_EM = load_data('batch_11_EM.csv', 'EM')
data_EM_LT = load_data('batch_11_EM_LT.csv', 'LT')

In [34]:
retrieve_all_alphas(dfs=[data_LT, data_LT_EM], names=['LT', 'EM'], batch=11)
retrieve_all_alphas(dfs=[data_EM, data_EM_LT], names=['EM', 'LT'], batch=11)

BATCH 11 // Krippendorff's alpha LT-EM:
[STRATEGY] 0.48
[GROUP] 0.33
[GOAL] 0.25
[HATE] 0.51
[TARGET] 0.57


BATCH 11 // Krippendorff's alpha EM-LT:
[STRATEGY] 0.35
[GROUP] 0.24
[GOAL] 0.21
[HATE] 0.39
[TARGET] 0.57




## Batch 12

In [35]:
data_LT = load_data('batch_12_LT.csv', 'LT')
data_LT_EM = load_data('batch_12_LT_EM.csv', 'EM')

data_EM = load_data('batch_12_EM.csv', 'EM')
data_EM_LT = load_data('batch_12_EM_LT.csv', 'LT')

In [36]:
retrieve_all_alphas(dfs=[data_LT, data_LT_EM], names=['LT', 'EM'], batch=12)
retrieve_all_alphas(dfs=[data_EM, data_EM_LT], names=['EM', 'LT'], batch=12)

BATCH 12 // Krippendorff's alpha LT-EM:
[STRATEGY] 0.34
[GROUP] 0.13
[GOAL] 0.07
[HATE] 0.56
[TARGET] 0.63


BATCH 12 // Krippendorff's alpha EM-LT:
[STRATEGY] 0.18
[GROUP] 0.17
[GOAL] 0.09
[HATE] 0.24
[TARGET] 0.29




## Batch 13

In [37]:
data_LT = load_data('batch_13_LT.csv', 'LT')
data_LT_EM = load_data('batch_13_LT_EM.csv', 'EM')

data_EM = load_data('batch_13_EM.csv', 'EM')
data_EM_LT = load_data('batch_13_EM_LT.csv', 'LT')

In [38]:
retrieve_all_alphas(dfs=[data_LT, data_LT_EM], names=['LT', 'EM'], batch=13)
retrieve_all_alphas(dfs=[data_EM, data_EM_LT], names=['EM', 'LT'], batch=13)

BATCH 13 // Krippendorff's alpha LT-EM:
[STRATEGY] 0.40
[GROUP] 0.11
[GOAL] 0.15
[HATE] 0.41
[TARGET] 0.46


BATCH 13 // Krippendorff's alpha EM-LT:
[STRATEGY] 0.36
[GROUP] 0.17
[GOAL] -0.08
[HATE] 0.45
[TARGET] 0.53




## Batch 14

In [39]:
data_EM = load_data('batch_14_EM.csv', 'EM')
data_EM_AH = load_data('batch_14_EM_AH.csv', 'AH')

In [40]:
retrieve_all_alphas(dfs=[data_EM, data_EM_AH], names=['EM', 'AH'], batch=14)

BATCH 14 // Krippendorff's alpha EM-AH:
[STRATEGY] 0.54
[GROUP] 0.40
[GOAL] 0.28
[HATE] 0.61
[TARGET] 0.78


